### BentoML Example
# Titanic Survival Prediction with LightBGM

This is a BentoML Demo Project demonstrating how to package and serve LightBGM model for production using BentoML.

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment.

Let's get started!
![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=ligthbgm&ea=lightbgm-tiantic-survival-prediction&dt=lightbgm-tiantic-survival-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install bentoml
!pip install lightgbm numpy pandas

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import bentoml

# Prepare Dataset
download dataset from https://www.kaggle.com/c/titanic/data

In [3]:
!mkdir data
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv -o ./data/train.csv
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv -o ./data/test.csv

mkdir: data: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60302  100 60302    0     0   154k      0 --:--:-- --:--:-- --:--:--  154k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28210  100 28210    0     0   101k      0 --:--:-- --:--:-- --:--:--  101k


In [4]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
train_df.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [5]:
y = train_df.pop('Survived')
cols = ['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']
X_train, X_test, y_train, y_test = train_test_split(train_df[cols], 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [6]:
# Create an LGBM dataset for training
train_data = lgb.Dataset(data=X_train[cols],
                        label=y_train)

# Create an LGBM dataset from the test
test_data = lgb.Dataset(data=X_test[cols],
                        label=y_test)

# Model Training

In [7]:
lgb_params = {
    'boosting': 'dart',          # dart (drop out trees) often performs better
    'application': 'binary',     # Binary classification
    'learning_rate': 0.05,       # Learning rate, controls size of a gradient descent step
    'min_data_in_leaf': 20,      # Data set is quite small so reduce this a bit
    'feature_fraction': 0.7,     # Proportion of features in each boost, controls overfitting
    'num_leaves': 41,            # Controls size of tree since LGBM uses leaf wise splits
    'metric': 'binary_logloss',  # Area under ROC curve as the evaulation metric
    'drop_rate': 0.15
              }

evaluation_results = {}
model = lgb.train(train_set=train_data,
                params=lgb_params,
                valid_sets=[train_data, test_data], 
                valid_names=['Train', 'Test'],
                evals_result=evaluation_results,
                num_boost_round=500,
                early_stopping_rounds=100,
                verbose_eval=20
                )

[20]	Train's binary_logloss: 0.55215	Test's binary_logloss: 0.587358
[40]	Train's binary_logloss: 0.510164	Test's binary_logloss: 0.559348
[60]	Train's binary_logloss: 0.500602	Test's binary_logloss: 0.551635
[80]	Train's binary_logloss: 0.490215	Test's binary_logloss: 0.547154
[100]	Train's binary_logloss: 0.486812	Test's binary_logloss: 0.547076
[120]	Train's binary_logloss: 0.479242	Test's binary_logloss: 0.542552
[140]	Train's binary_logloss: 0.469847	Test's binary_logloss: 0.539319
[160]	Train's binary_logloss: 0.471384	Test's binary_logloss: 0.542278
[180]	Train's binary_logloss: 0.453052	Test's binary_logloss: 0.535512
[200]	Train's binary_logloss: 0.442048	Test's binary_logloss: 0.533921
[220]	Train's binary_logloss: 0.436788	Test's binary_logloss: 0.534261
[240]	Train's binary_logloss: 0.427196	Test's binary_logloss: 0.532026
[260]	Train's binary_logloss: 0.420145	Test's binary_logloss: 0.531791
[280]	Train's binary_logloss: 0.413336	Test's binary_logloss: 0.527412
[300]	Train

In [8]:
test_df['pred'] = model.predict(test_df[cols])
test_df[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)

Pclass   Age     Fare  SibSp  Parch      pred
10       3   NaN   7.8958      0      0  0.052353
11       1  46.0  26.0000      0      0  0.308877

## Create BentoService for model serving

In [9]:
%%writefile lightbgm_titanic_bento_service.py

import lightgbm as lgb

import bentoml
from bentoml.artifact import LightGBMModelArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([LightGBMModelArtifact('model')])
@bentoml.env(pip_dependencies=['lightgbm'])
class TitanicSurvivalPredictionService(bentoml.BentoService):
    
    @bentoml.api(DataframeHandler)
    def predict(self, df):
        data = df[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']]
        return self.artifacts.model.predict(data)

Overwriting lightbgm_titanic_bento_service.py


# Save BentoML service archive

In [10]:
# 1) import the custom BentoService defined above
from lightbgm_titanic_bento_service import TitanicSurvivalPredictionService

# 2) `pack` it with required artifacts
bento_service = TitanicSurvivalPredictionService()
bento_service.pack('model', model)

# 3) save your BentoSerivce
saved_path = bento_service.save()

[2020-01-22 15:41:16,540] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-01-22 15:41:16,861] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'examples'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'docs'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check


creating BentoML-0.5.8+23.g1dd72d3
creating BentoML-0.5.8+23.g1dd72d3/BentoML.egg-info
creating BentoML-0.5.8+23.g1dd72d3/bentoml
creating BentoML-0.5.8+23.g1dd72d3/bentoml/artifact
creating BentoML-0.5.8+23.g1dd72d3/bentoml/bundler
creating BentoML-0.5.8+23.g1dd72d3/bentoml/cli
creating BentoML-0.5.8+23.g1dd72d3/bentoml/clipper
creating BentoML-0.5.8+23.g1dd72d3/bentoml/configuration
creating BentoML-0.5.8+23.g1dd72d3/bentoml/deployment
creating BentoML-0.5.8+23.g1dd72d3/bentoml/deployment/aws_lambda
creating BentoML-0.5.8+23.g1dd72d3/bentoml/deployment/sagemaker
creating BentoML-0.5.8+23.g1dd72d3/bentoml/handlers
creating BentoML-0.5.8+23.g1dd72d3/bentoml/migrations
creating BentoML-0.5.8+23.g1dd72d3/bentoml/migrations/versions
creating BentoML-0.5.8+23.g1dd72d3/bentoml/proto
creating BentoML-0.5.8+23.g1dd72d3/bentoml/repository
creating BentoML-0.5.8+23.g1dd72d3/bentoml/server
creating BentoML-0.5.8+23.g1dd72d3/bentoml/server/static
creating BentoML-0.5.8+23.g1dd72d3/bentoml/utils
c

copying bentoml/repository/__init__.py -> BentoML-0.5.8+23.g1dd72d3/bentoml/repository
copying bentoml/repository/metadata_store.py -> BentoML-0.5.8+23.g1dd72d3/bentoml/repository
copying bentoml/server/__init__.py -> BentoML-0.5.8+23.g1dd72d3/bentoml/server
copying bentoml/server/bento_api_server.py -> BentoML-0.5.8+23.g1dd72d3/bentoml/server
copying bentoml/server/bento_sagemaker_server.py -> BentoML-0.5.8+23.g1dd72d3/bentoml/server
copying bentoml/server/gunicorn_config.py -> BentoML-0.5.8+23.g1dd72d3/bentoml/server
copying bentoml/server/gunicorn_server.py -> BentoML-0.5.8+23.g1dd72d3/bentoml/server
copying bentoml/server/middlewares.py -> BentoML-0.5.8+23.g1dd72d3/bentoml/server
copying bentoml/server/utils.py -> BentoML-0.5.8+23.g1dd72d3/bentoml/server
copying bentoml/server/static/swagger-ui-bundle.js -> BentoML-0.5.8+23.g1dd72d3/bentoml/server/static
copying bentoml/server/static/swagger-ui.css -> BentoML-0.5.8+23.g1dd72d3/bentoml/server/static
copying bentoml/utils/__init__.py

## Load saved BentoService for serving


In [15]:
import bentoml

bento_model = bentoml.load(saved_path)

result = bento_model.predict(test_df)
test_df['pred'] = result
test_df[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)

[2019-11-25 23:58:54,631] WARNING - Module `lightbgm_titanic_bento_service` already loaded, using existing imported module.


Pclass   Age     Fare  SibSp  Parch      pred
10       3   NaN   7.8958      0      0  0.052353
11       1  46.0  26.0000      0      0  0.308877

# Access BentoService bundle with CLI

In [11]:
!bentoml get TitanicSurvivalPredictionService

BENTO_SERVICE                                           CREATED_AT        URI                                                                                        APIS                       ARTIFACTS
TitanicSurvivalPredictionService:20200122154116_FCEE7A  2020-01-22 23:41  /Users/bozhaoyu/bentoml/repository/TitanicSurvivalPredictionService/20200122154116_FCEE7A  predict::DataframeHandler  model::LightGBMModelArtifact


In [13]:
!bentoml get {bento_service.name}:{bento_service.version}

{
  "name": "TitanicSurvivalPredictionService",
  "version": "20200122154116_FCEE7A",
  "uri": {
    "type": "LOCAL",
    "uri": "/Users/bozhaoyu/bentoml/repository/TitanicSurvivalPredictionService/20200122154116_FCEE7A"
  },
  "bentoServiceMetadata": {
    "name": "TitanicSurvivalPredictionService",
    "version": "20200122154116_FCEE7A",
    "createdAt": "2020-01-22T23:41:33.581979Z",
    "env": {
      "condaEnv": "name: bentoml-TitanicSurvivalPredictionService\nchannels:\n- defaults\ndependencies:\n- python=3.7.3\n- pip\n",
      "pipDependencies": "bentoml==0.5.8\nlightgbm",
      "pythonVersion": "3.7.3"
    },
    "artifacts": [
      {
        "name": "model",
        "artifactType": "LightGBMModelArtifact"
      }
    ],
    "apis": [
      {
        "name": "predict",
        "handlerType": "DataframeHandler",
        "docs": "BentoService API"
      }
    ]
  }
}


In [22]:
!bentoml run predict /Users/bozhaoyu/bentoml/repository/TitanicSurvivalPredictionService/20200122154116_FCEE7A \
--input '{"PassengerId":{"3":895},"Pclass":{"3":3},"Name":{"3":"Wirz, Mr. Albert"},"Sex":{"3":"male"},"Age":{"3":27.0},"SibSp":{"3":0},"Parch":{"3":0},"Ticket":{"3":"315154"},"Fare":{"3":8.6625},"Cabin":{"3":null},"Embarked":{"3":"S"},"pred":{"3":0.5045963287}}'

[2020-01-22 16:58:19,400] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-01-22 16:58:19,412] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.8,  but loading from BentoML version 0.5.8+23.g1dd72d3
[2020-01-22 16:58:20,036] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will b

## BentoService Serving via REST API

In your termnial, run the following command to start the REST API server:

In [32]:
!bentoml serve {saved_path}

[2020-01-22 17:22:12,985] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-01-22 17:22:13,000] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.8,  but loading from BentoML version 0.5.8+23.g1dd72d3
[2020-01-22 17:22:14,210] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will b

Open http://127.0.0.1:5000 to see more information about the REST APIs server in your
browser.


### Send prediction requeset to the REST API server

Navigate to parent directory of the notebook(so you have reference to the `test.jpg` image), and run the following `curl` command to send the image to REST API server and get a prediction result:

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]' \
localhost:5000/predict
```

## Use BentoService as PyPI package

In [33]:
! pip install {saved_path}

Processing /Users/bozhaoyu/bentoml/repository/TitanicSurvivalPredictionService/20200122154116_FCEE7A


  Created wheel for TitanicSurvivalPredictionService: filename=TitanicSurvivalPredictionService-20200122154116_FCEE7A-cp37-none-any.whl size=643993 sha256=43f209cae82f18fda0ee3ed740e3380e7dad0137589f2ec5209519a01d732950
  Stored in directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/pip-ephem-wheel-cache-l_r9ibhg/wheels/cd/9a/19/afbde887befa6ee8aa83fde19cf1dbb91cacc26bb782362185
Successfully built TitanicSurvivalPredictionService


In [34]:
!TitanicSurvivalPredictionService info

[2020-01-22 18:21:12,485] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-01-22 18:21:12,494] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.8,  but loading from BentoML version 0.5.8+23.g1dd72d3
{
  "name": "TitanicSurvivalPredictionService",
  "version": "20200122154116_FCEE7A",
  "created_at": "2020-01-22T23:41:33.581979Z",
  "env": {
    "conda_env": "name: bentoml-TitanicSurvivalPredictionService\nchannels:\n- defa

In [35]:
!TitanicSurvivalPredictionService run predict --input '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]' 

[2020-01-22 18:21:34,109] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-01-22 18:21:34,119] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.8,  but loading from BentoML version 0.5.8+23.g1dd72d3
[2020-01-22 18:21:34,122] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will b

# Containerize REST API server with Docker


The BentoService SavedBundle is structured to work as a docker build context, that can be directed used to build a docker image for API server. Simply use it as the docker build context directory:

In [36]:
!cd {saved_path} && docker build -t lightgbm-titanic .

Sending build context to Docker daemon  2.407MB
Step 1/12 : FROM continuumio/miniconda3:4.7.12
 ---> 406f2b43ea59
Step 2/12 : ENTRYPOINT [ "/bin/bash", "-c" ]
 ---> Using cache
 ---> d3cfb80f9fc7
Step 3/12 : EXPOSE 5000
 ---> Using cache
 ---> 3d1f997a1d9c
Step 4/12 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> a0b2d59e3e13
Step 5/12 : RUN conda install pip numpy scipy       && pip install gunicorn
 ---> Using cache
 ---> 504b4d11f51a
Step 6/12 : COPY . /bento
 ---> 51b0475d6d63
Step 7/12 : WORKDIR /bento
 ---> Running in a0a1120a1219
Removing intermediate container a0a1120a1219
 ---> dcb8182dbfeb
Step 8/12 : RUN if [ -f /bento/setup.sh ]; then /bin/bash -c /bento/setup.sh; fi
 ---> Running in 0bde88c000ec
Removing intermediate container 0bde88c000ec
 ---> 2cefb754e02b
Step 9/12 : RUN conda env update -n base -f /bento/environment.yml
 ---

  Created wheel for python-json-logger: filename=python_json_logger-0.1.11-py2.py3-none-any.whl size=5076 sha256=2271c4e4cb6933374b1494f071ea3f78a7b395063b9b40d1e5f1740ab574ef6c
  Stored in directory: /root/.cache/pip/wheels/97/f7/a1/752e22bb30c1cfe38194ea0070a5c66e76ef4d06ad0c7dc401
  Created wheel for prometheus-client: filename=prometheus_client-0.7.1-cp37-none-any.whl size=41402 sha256=10031561bb8551f2df3d0a8a2f9abd89f4ecd48c249eb31017576c9ab033e032
  Stored in directory: /root/.cache/pip/wheels/1c/54/34/fd47cd9b308826cc4292b54449c1899a30251ef3b506bc91ea
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.13-cp37-cp37m-linux_x86_64.whl size=1223698 sha256=834075adcd2a5ec4bf74e282e128812c8eaeef0166f1990d0e3e399d4d9e7b06
  Stored in directory: /root/.cache/pip/wheels/b3/35/98/4c9cb3fd63d21d5606b972dd70643769745adf60e622467b71
  Created wheel for cerberus: filename=Cerberus-1.3.2-cp37-none-any.whl size=54336 sha256=56428c7a4f10f87f9ac7c4d1e2250641475dab5cd4ca129d3db6f1d6cee64b5d
 

  Created wheel for BentoML: filename=BentoML-0.5.8+23.g1dd72d3-cp37-none-any.whl size=505485 sha256=ace292105b8bb6c2e2195909a4e5534ccbaaeb43f830320a0b8e672127078966
  Stored in directory: /root/.cache/pip/wheels/60/9d/9e/5f9949596e00855daaa388e2c0777603f23e7cf4c6b2708be7
Successfully built BentoML
  Found existing installation: BentoML 0.5.8
    Uninstalling BentoML-0.5.8:
      Successfully uninstalled BentoML-0.5.8
Removing intermediate container 1aed15a9ff22
 ---> 6c157daf4299
Step 12/12 : CMD ["bentoml serve-gunicorn /bento"]
 ---> Running in d6af172a1af9
Removing intermediate container d6af172a1af9
 ---> c1ff3dcf6ccb
Successfully built c1ff3dcf6ccb
Successfully tagged lightgbn-titanic:latest


Next, you can docker push the image to your choice of registry for deployment, or run it locally for development and testing:

In [ ]:
!docker run -p 5000:5000 lightgbm-titanic

# Deploy BentoService as REST API server to the cloud


BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html

For this demo, we are going to deploy to AWS Sagemaker

**Deploying to Sagemaker with `bentoml sagemaker deploy` command**

In [24]:
!bentoml sagemaker deploy first-light-bgm -b {bento_service.name}:{bento_service.version} \
    --api-name predict --region us-west-2

[2020-01-22 16:59:41,021] DEBUG - Using BentoML with local Yatai server
[2020-01-22 16:59:41,106] DEBUG - Upgrading tables to the latest revision
Deploying Sagemaker deployment /[2020-01-22 16:59:42,117] DEBUG - Created temporary directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-esrezr4x
\[2020-01-22 16:59:42,748] DEBUG - Getting docker login info from AWS
[2020-01-22 16:59:42,749] DEBUG - Building docker image: 192023623294.dkr.ecr.us-west-2.amazonaws.com/titanicsurvivalpredictionservice-sagemaker:20200122154116_FCEE7A
-[2020-01-22 16:59:43,159] INFO - Step 1/11 : FROM continuumio/miniconda3:4.7.12
[2020-01-22 16:59:43,159] INFO - 

[2020-01-22 16:59:43,160] INFO -  ---> 406f2b43ea59

[2020-01-22 16:59:43,160] INFO - Step 2/11 : EXPOSE 8080
[2020-01-22 16:59:43,160] INFO - 

[2020-01-22 16:59:43,160] INFO -  ---> Using cache

[2020-01-22 16:59:43,160] INFO -  ---> d8ec94af8603

[2020-01-22 16:59:43,160] INFO - Step 3/11 : RUN set -x      && apt-get upd

-[2020-01-22 17:00:16,339] INFO - Collecting pandas

[2020-01-22 17:00:16,376] INFO -   Downloading https://files.pythonhosted.org/packages/63/e0/a1b39cdcb2c391f087a1538bc8a6d62a82d0439693192aef541d7b123769/pandas-0.25.3-cp37-cp37m-manylinux1_x86_64.whl (10.4MB)

/[2020-01-22 17:00:26,234] INFO - Collecting flask

[2020-01-22 17:00:26,266] INFO -   Downloading https://files.pythonhosted.org/packages/9b/93/628509b8d5dc749656a9641f4caf13540e2cdec85276964ff8f43bbb1d3b/Flask-1.1.1-py2.py3-none-any.whl (94kB)

|[2020-01-22 17:00:26,375] INFO - Requirement already satisfied: requests in /opt/conda/lib/python3.7/site-packages (from bentoml==0.5.8->-r /opt/program/requirements.txt (line 1)) (2.22.0)

\[2020-01-22 17:00:26,445] INFO - Collecting python-dateutil<2.8.1,>=2.1

[2020-01-22 17:00:26,480] INFO -   Downloading https://files.pythonhosted.org/packages/41/17/c62faccbfbd163c7f57f3844689e3a78bae1f403648a6afb1d0866d87fbb/python_dateutil-2.8.0-py2.py3-none-any.whl (226kB)

/[2020-01-22 

\[2020-01-22 17:01:03,257] INFO -   Building wheel for python-json-logger (setup.py): finished with status 'done'

[2020-01-22 17:01:03,258] INFO -   Created wheel for python-json-logger: filename=python_json_logger-0.1.11-py2.py3-none-any.whl size=5076 sha256=4a000ec088dd0d565eed9ed4e8107d6a9823825d2441c35fabb475d67c73d60e

[2020-01-22 17:01:03,258] INFO -   Stored in directory: /root/.cache/pip/wheels/97/f7/a1/752e22bb30c1cfe38194ea0070a5c66e76ef4d06ad0c7dc401

[2020-01-22 17:01:03,263] INFO -   Building wheel for tabulate (setup.py): started

/[2020-01-22 17:01:03,534] INFO -   Building wheel for tabulate (setup.py): finished with status 'done'

[2020-01-22 17:01:03,534] INFO -   Created wheel for tabulate: filename=tabulate-0.8.6-cp37-none-any.whl size=23274 sha256=662ecb34c9909bfe354e49d231a1691d1089431556c34e7bbc22e252d53f0fd7
  Stored in directory: /root/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e

[2020-01-22 17:01:03,539] INFO -   Building w

-[2020-01-22 17:05:29,627] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfigName': 'dev-first-light--TitanicSurvivalPredi-20200122154116-FCEE7A', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '210d4b96-5987-4036-84fe-4135e1bcfabd', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '210d4b96-5987-4036-84fe-4135e1bcfabd', 'content-type': 'application/x-amz-json-1.1', 'content-length': '303', 'date': 'Thu, 23 Jan 2020 01:05:29 GMT'}, 'RetryAttempts': 0}}
\[2020-01-22 17:05:34,855] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfi

|[2020-01-22 17:06:27,764] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfigName': 'dev-first-light--TitanicSurvivalPredi-20200122154116-FCEE7A', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '8c7667d8-14ce-4fef-9bbf-dfd9f8257985', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8c7667d8-14ce-4fef-9bbf-dfd9f8257985', 'content-type': 'application/x-amz-json-1.1', 'content-length': '303', 'date': 'Thu, 23 Jan 2020 01:06:27 GMT'}, 'RetryAttempts': 0}}
/[2020-01-22 17:06:32,972] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfi

...

-[2020-01-22 17:08:23,619] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfigName': 'dev-first-light--TitanicSurvivalPredi-20200122154116-FCEE7A', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'f11ed820-aedb-4959-b465-7c9aecede1a8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f11ed820-aedb-4959-b465-7c9aecede1a8', 'content-type': 'application/x-amz-json-1.1', 'content-length': '303', 'date': 'Thu, 23 Jan 2020 01:08:23 GMT'}, 'RetryAttempts': 0}}
\[2020-01-22 17:08:28,834] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfi

/[2020-01-22 17:09:21,058] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfigName': 'dev-first-light--TitanicSurvivalPredi-20200122154116-FCEE7A', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '581e6d5d-537d-42b4-b4d4-f5f22cc25ac4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '581e6d5d-537d-42b4-b4d4-f5f22cc25ac4', 'content-type': 'application/x-amz-json-1.1', 'content-length': '303', 'date': 'Thu, 23 Jan 2020 01:09:20 GMT'}, 'RetryAttempts': 0}}
-[2020-01-22 17:09:26,286] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfi

|[2020-01-22 17:10:18,494] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfigName': 'dev-first-light--TitanicSurvivalPredi-20200122154116-FCEE7A', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'c7110e0f-830b-4388-bdc1-d39aa7b8793f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c7110e0f-830b-4388-bdc1-d39aa7b8793f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '303', 'date': 'Thu, 23 Jan 2020 01:10:17 GMT'}, 'RetryAttempts': 0}}
/[2020-01-22 17:10:23,714] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfi

-[2020-01-22 17:11:16,138] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfigName': 'dev-first-light--TitanicSurvivalPredi-20200122154116-FCEE7A', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'f5322a60-4478-4d7c-901e-c504ee1723ed', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f5322a60-4478-4d7c-901e-c504ee1723ed', 'content-type': 'application/x-amz-json-1.1', 'content-length': '303', 'date': 'Thu, 23 Jan 2020 01:11:15 GMT'}, 'RetryAttempts': 0}}
\[2020-01-22 17:11:21,351] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfi

-[2020-01-22 17:12:13,576] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfigName': 'dev-first-light--TitanicSurvivalPredi-20200122154116-FCEE7A', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 1, 22, 17, 5, 24, 56000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '4f868501-ca68-45c2-b132-f3c9ee846851', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4f868501-ca68-45c2-b132-f3c9ee846851', 'content-type': 'application/x-amz-json-1.1', 'content-length': '303', 'date': 'Thu, 23 Jan 2020 01:12:12 GMT'}, 'RetryAttempts': 0}}
\[2020-01-22 17:12:18,783] DEBUG - AWS describe endpoint response: {'EndpointName': 'dev-first-light-bgm', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm', 'EndpointConfi

** Use `bentoml sagemaker get DEPLOYMENT_NAME` to get latest state and other information such as endpoint**

In [25]:
!bentoml sagemaker get first-light-bgm

{
  "namespace": "dev",
  "name": "first-light-bgm",
  "spec": {
    "bentoName": "TitanicSurvivalPredictionService",
    "bentoVersion": "20200122154116_FCEE7A",
    "operator": "AWS_SAGEMAKER",
    "sagemakerOperatorConfig": {
      "region": "us-west-2",
      "instanceType": "ml.m4.xlarge",
      "instanceCount": 1,
      "apiName": "predict"
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "EndpointName": "dev-first-light-bgm",
      "EndpointArn": "arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm",
      "EndpointConfigName": "dev-first-light--TitanicSurvivalPredi-20200122154116-FCEE7A",
      "ProductionVariants": [
        {
          "VariantName": "dev-first-light--TitanicSurvivalPredi-20200122154116-FCEE7A",
          "DeployedImages": [
            {
              "SpecifiedImage": "192023623294.dkr.ecr.us-west-2.amazonaws.com/titanicsurvivalpredictionservice-sagemaker:20200122154116_FCEE7A",
              "ResolvedImage": "192023

Test our sagemaker deployment with a sample data

In [29]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name dev-first-light-bgm \
  --body '{"PassengerId":{"3":895},"Pclass":{"3":3},"Name":{"3":"Wirz, Mr. Albert"},"Sex":{"3":"male"},"Age":{"3":27.0},"SibSp":{"3":0},"Parch":{"3":0},"Ticket":{"3":"315154"},"Fare":{"3":8.6625},"Cabin":{"3":null},"Embarked":{"3":"S"},"pred":{"3":0.5045963287}}' \
  --content-type "application/json" \
  output.json && cat output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "dev-first-light--TitanicSurvivalPredi-20200122154116-FCEE7A"
}
[0.5045963286563061]

**`bentoml sagemaker list` command provides an overview of all of the Sagemaker deployments**

In [30]:
!bentoml sagemaker list

NAME             NAMESPACE    LABELS    PLATFORM       STATUS    AGE
first-light-bgm  dev                    aws-sagemaker  running   14 minutes and 37 seconds


**`bentoml sagemaker delete` deletes the sagemaker deployment and related resources on AWS**

In [31]:
!bentoml sagemaker delete first-light-bgm

Successfully deleted AWS Sagemaker deployment "first-light-bgm"
